# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle
from src.match_names import name_fbref_to_fpl, neutralize_name
from config import *

In [3]:
info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]

In [4]:
FEATURES_OUTFIELD = ['Was_home',
 'Rating_difference',
 'Avg_FPL_points',
 'xP_4',
 'DEF',
 'FWD',
 'MID']

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (2,3,6,36,43,44,45,59,84,86,147,208) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
# df[FEATURES_OUTFIELD].isna().sum()

In [7]:
df = df[~df[FEATURES_OUTFIELD].isnull().any(axis=1)] # for some reason 45 rows are missing some feature values

In [8]:
X_test = df[df["Season"] == CURRENT_SEASON][FEATURES_OUTFIELD]

In [9]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30', 'Min_points_30',
       'Team_result_30', 'DEF', 'FWD', 'GK', 'MID', 'Finished'],
      dtype='object', length=274)

In [10]:
X_test.columns

Index(['Was_home', 'Rating_difference', 'Avg_FPL_points', 'xP_4', 'DEF', 'FWD',
       'MID'],
      dtype='object')

In [11]:
# df[df["FPL_pos"] == "GK"]

In [12]:
file = open("models/GBR.pkl",'rb')
model = pickle.load(file)

In [13]:
model

LinearRegression()

# Predicting

In [14]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [15]:
def adjust_goalkeepers(df):
    gks = pd.read_csv("data/misc/goalkeepers.csv")
    gk_names = gks["Name"].to_list()
    gk_names = [name_fbref_to_fpl(neutralize_name(n)) for n in gk_names]
    # print(gk_names)
    df.loc[(df["FPL_pos"] == "GK") & ~(df["Name_original"].apply(neutralize_name).isin(gk_names)), "Pred"] = 0
    return df

In [16]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_goalkeepers(preds)
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [17]:
preds = get_predictions(model, df, X_test)

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [18]:
# preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == NEXT_GAMEWEEK)].head(30)

In [28]:
# next gameweek
preds[preds["GW"] == NEXT_GAMEWEEK].head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
5514,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,FWD,14.0,7.033665
12105,Mohamed Salah,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,MID,12.5,6.794331
6316,Guglielmo Vicario,4.0,2023-24,Tottenham,Burnley,0.0,1829.239014,1722.952881,GK,5.0,6.098498
3314,Carlton Morris,4.0,2023-24,Luton,West Ham,1.0,1602.477417,1787.825562,FWD,5.5,6.035774
3000,Bryan Mbeumo,4.0,2023-24,Brentford,Bournemouth,1.0,1835.573242,1660.290894,MID,6.7,5.847816
2930,Bruno Borges Fernandes,4.0,2023-24,Manchester Utd,Arsenal,0.0,1865.755249,1927.032104,MID,8.5,5.496702
13363,Phil Foden,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,MID,7.6,5.338258
15802,Trent Alexander-Arnold,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,DEF,7.9,5.102164
7924,Jayden Bogle,4.0,2023-24,Sheffield United,Everton,1.0,1631.764282,1691.463989,DEF,4.5,5.090329
7784,Jarrod Bowen,4.0,2023-24,West Ham,Luton,0.0,1787.825562,1602.477417,MID,7.0,5.026571


In [21]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
5542,Erling Haaland,33.0,2023-24,Manchester City,Luton,1.0,2083.702148,1602.477417,FWD,14.0,7.173734
5529,Erling Haaland,20.0,2023-24,Manchester City,Sheffield United,1.0,2083.702148,1631.764282,FWD,14.0,7.140270
636,Mohamed Salah,2.0,2023-24,Liverpool,Bournemouth,1.0,1943.976074,1662.536621,MID,12.5,7.110498
5521,Erling Haaland,11.0,2023-24,Manchester City,Bournemouth,1.0,2083.702148,1660.290894,FWD,14.0,7.108809
5516,Erling Haaland,6.0,2023-24,Manchester City,Nott'ham Forest,1.0,2083.702148,1677.877930,FWD,14.0,7.089946
5533,Erling Haaland,24.0,2023-24,Manchester City,Everton,1.0,2083.702148,1691.463989,FWD,14.0,7.075643
5545,Erling Haaland,36.0,2023-24,Manchester City,Wolves,1.0,2083.702148,1704.537109,FWD,14.0,7.062095
635,Mohamed Salah,1.0,2023-24,Liverpool,Chelsea,0.0,1946.899780,1788.094360,MID,12.5,7.053823
5531,Erling Haaland,22.0,2023-24,Manchester City,Burnley,1.0,2083.702148,1722.952881,FWD,14.0,7.043360
5514,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,FWD,14.0,7.033665


In [22]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Mohamed Salah,253.591894
Erling Haaland,251.295017
Guglielmo Vicario,233.833192
Bruno Borges Fernandes,220.056604
Carlton Morris,216.124311
Bryan Mbeumo,204.522095
Jarrod Bowen,191.154589
Trent Alexander-Arnold,189.809719
Phil Foden,188.549665


# Saving to file

In [23]:
preds.to_csv("predictions/preds_next_season.csv", index=False)

# Importance

In [24]:
# import shap
# explainer = shap.Explainer(model.predict, X_test[9000:10000])
# shap_values = explainer(X_test[9000:10000])

In [25]:
# shap.plots.bar(shap_values, max_display=15)

In [26]:
# # Salah vs Nottingham Forest
# shap.plots.waterfall(shap_values[502])

In [27]:
# shap.plots.waterfall(shap_values[470])